<center>Progetto realizzato da Elena Curti (matr. 185431)

# Crimes
</center>

## Introduzione

## Requisiti


## Operazioni iniziali

In [1]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "password1234"))
print("Connessione al database eseguita correttamente!")


Connessione al database eseguita correttamente!


## Descrizione del database

## 1. Trovare i gruppi di criminali

In [2]:
## Creo il project
NOME_GRAPH_PROJECT = "criminals"
if graph.run("call gds.graph.exists('"+NOME_GRAPH_PROJECT+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_PROJECT+'")')

cq = """ 
  CALL gds.graph.project.cypher('""" + NOME_GRAPH_PROJECT+ """',
  ' MATCH (p:Person) WHERE 
        EXISTS ( (p)-[:PARTY_TO]->(:Crime) )                          // Persone che hanno preso parte a un crimine
        //OR EXISTS ( (p)-[:KNOWS]-(:Person)-[:PARTY_TO]->(:Crime) )    // Conoscenti dei criminali
    RETURN id(p) as id',
  'MATCH (s:Person)-[:KNOWS]-(t:Person) RETURN id(s) AS source, id(t) AS target',
  {validateRelationships:false}
); 
"""
graph.run(cq)
print("Proiezione crata correttamente!")

Proiezione crata correttamente!


### Algoritmo di Louvain

In [3]:
# Memory estimation
graph.run(""" 
CALL gds.louvain.write.estimate('""" + NOME_GRAPH_PROJECT+ """', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory ; """ 
)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
29,30,7153,566272,[7153 Bytes ... 553 KiB]


In [4]:
cq = """ 
CALL gds.louvain.stream('""" + NOME_GRAPH_PROJECT+ """')
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, collect(node.name + " " + node.surname + " (" + node.nhs_no+")") AS gruppo
WHERE size(gruppo)>1
RETURN *
"""
def print_in_columns(my_list):
    columns = 5 #if len(my_list)>10 else 3
    len_max = str(max([len(i) for i in my_list]) + 4)
    for i in range(0, len(my_list), columns):
        print("  ", end="")
        [eval('print(f"{i:'+len_max+'}", end="")') for i in my_list[i:i+columns]]
        print("")

def print_id_gruppo(query_results):
  for (communityId, gruppo) in query_results:
      print("---------- Gruppo "+str(communityId)+" ----------")
      print_in_columns(gruppo)
      print("")


print_id_gruppo(graph.run(cq))

---------- Gruppo 27 ----------
  Raymond Walker (879-22-8665)        Kathleen Peters (250-75-5238)       Diana Murray (900-41-3309)          Phillip Williamson (337-28-4424)    Jessica Kelly (311-75-6483)         

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Jack Powell (249-54-6589)      Brian Morales (335-71-7747)    

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Donald Robinson (873-28-7561)      Andrea Montgomery (351-83-4608)    



### Label Propagation


In [5]:
graph.run(""" 
CALL gds.labelPropagation.write.estimate('""" + NOME_GRAPH_PROJECT+ """', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
""" )

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
29,30,2288,5872,[2288 Bytes ... 5872 Bytes]


In [6]:
cq = """ 
CALL gds.labelPropagation.stream('""" + NOME_GRAPH_PROJECT+ """')
YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) as node, communityId
WITH communityId, collect(node.name + " " + node.surname + " (" + node.nhs_no+")") AS gruppo
WHERE size(gruppo)>1
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 863 ----------
  Raymond Walker (879-22-8665)        Kathleen Peters (250-75-5238)       Diana Murray (900-41-3309)          Alan Ward (881-20-2396)             Jack Powell (249-54-6589)           
  Phillip Williamson (337-28-4424)    Jessica Kelly (311-75-6483)         Brian Morales (335-71-7747)         

---------- Gruppo 1144 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 1198 ----------
  Donald Robinson (873-28-7561)      Andrea Montgomery (351-83-4608)    



### Weakly Connected Components

In [7]:
cq = """ 
CALL gds.beta.modularityOptimization.stream('""" + NOME_GRAPH_PROJECT+ """')
YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) as node, communityId
WITH communityId, collect(node.name + " " + node.surname + " (" + node.nhs_no+")") AS gruppo
WHERE size(gruppo)>1
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 14 ----------
  Raymond Walker (879-22-8665)        Kathleen Peters (250-75-5238)       Phillip Williamson (337-28-4424)    

---------- Gruppo 27 ----------
  Diana Murray (900-41-3309)     Jessica Kelly (311-75-6483)    

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Jack Powell (249-54-6589)      Brian Morales (335-71-7747)    

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Donald Robinson (873-28-7561)      Andrea Montgomery (351-83-4608)    



### K-1 Coloring

In [8]:
cq = """ 
CALL gds.beta.k1coloring.stream('""" + NOME_GRAPH_PROJECT+ """')
YIELD nodeId, color
WITH gds.util.asNode(nodeId) as node, color
WITH color, collect(node.name + " " + node.surname + " (" + node.nhs_no+")") AS gruppo
WHERE size(gruppo)>1
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 0 ----------
  Stephanie Hughes (821-11-2735)      Maria Hughes (678-06-9352)          David Mills (589-69-0106)           Rebecca Long (785-79-1645)          Annie George (575-05-6519)          
  Michelle Patterson (600-92-0643)    Norma Payne (699-85-1673)           Amanda Robertson (455-19-0708)      Gary Vasquez (679-81-5309)          Fred Williamson (468-82-3915)       
  Carlos Black (859-81-0332)          Ernest Clark (205-52-5697)          Amy Bailey (276-19-9235)            Raymond Walker (879-22-8665)        Diana Murray (900-41-3309)          
  Alan Ward (881-20-2396)             Kenneth Carroll (252-29-4929)       Billy Moore (846-48-9238)           Donald Robinson (873-28-7561)       Joan Flores (247-72-6304)           
  Victor Harper (220-62-1837)         Craig Marshall (890-58-5813)        

---------- Gruppo 1 ----------
  Kathleen Peters (250-75-5238)      Jack Powell (249-54-6589)          Lillian Martinez (397-28-4474)     Andrea Montgomery (351-

### Conclusioni
...

## 2. Quali sono gli agenti di polizia più importanti?


### Cypher query

In [17]:
def print_n_results(cq, n=100):
    ris = graph.run(cq)
    ris.sample_size = n
    display(ris)

cq = """ 
MATCH (o:Officer)<-[:INVESTIGATED_BY]-(c:Crime) 
WITH o, count(c) as num_invest
RETURN o.name as nome, o.surname as cognome, num_invest
ORDER BY num_invest DESC
LIMIT 10 ;
"""
print_n_results(cq)

nome,cognome,num_invest
Madelon,DeSousa,50
Cloe,Ings,47
Kania,Notti,46
Worthy,Nettles,45
Ricca,Miskimmon,44
Olenolin,Klehyn,44
Winonah,Skynner,44
Simmonds,Greensall,43
Hollyanne,Orcott,43
Hedy,Vinson,43


### PageRank

In [20]:
cq=""" 
CALL gds.pageRank.stream('officerss')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score AS pageRank
RETURN n.name as nome, n.surname as cognome, pageRank
ORDER BY pageRank DESC
LIMIT 10 ;
"""
print_n_results(cq)

nome,cognome,pageRank
Madelon,DeSousa,6.525000000000007
Cloe,Ings,6.142500000000005
Kania,Notti,6.015000000000005
Worthy,Nettles,5.887500000000005
Winonah,Skynner,5.760000000000004
Ricca,Miskimmon,5.760000000000004
Olenolin,Klehyn,5.760000000000004
Simmonds,Greensall,5.632500000000004
Hollyanne,Orcott,5.632500000000004
Hedy,Vinson,5.632500000000004
